In [11]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from TreeConstructor import *

import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn.init as init

import numpy as np

import random
from queue import *

from seq2seqLoader import *

import matplotlib
matplotlib.use("Agg")
import matplotlib.pylab as plt
from matplotlib.pyplot import imshow


from zss import simple_distance

In [12]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
class Tree:
    def __init__(self,level,rootObj=None):
        self.level = level
        #Label same as key. Just using a differnet attribute as in one tree key is a Variable
        self.label = rootObj
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None
        self.parent = None
        self.sibling = None
        self.children = [] 
        self.hiddenA = None
        self.hiddenF = None
        
    @staticmethod
    def get_children(node):
        return node.children

    @staticmethod
    def get_label(node):
        return node.label

In [14]:
train_filename = "/data2/t2t/synth_data/relabel/test.orig"
target_filename = "/data2/t2t/synth_data/relabel/test.interior_relabel"
data_loader = T2TDataLoader(train_filename,target_filename)
test_data = data_loader.get_data()
print(len(test_data))
print(test_data[0][0])
print(test_data[0][1])

1000
[111, 109, 109, 18, 47, 45, 110, 84, 109, 109, 75, 87, 89, 110, 0, 88, 110, 110, 112]
[115, 113, 113, 18, 110, 45, 114, 111, 113, 113, 75, 111, 89, 114, 109, 88, 114, 114, 116]


In [5]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, embedding_size):
        super(EncoderLSTM, self).__init__()
        self.word_embeddings = nn.Embedding(input_size, embedding_size)    
        self.lstm = nn.LSTM(input_size = embedding_size, hidden_size = hidden_size,num_layers = 1)
        

    def forward(self, input, hidden, c):
        embedded = self.word_embeddings(input).view(1, 1, -1)

        output = embedded
        output,(hidden,c) = self.lstm(output, (hidden, c))
        return output,hidden,c

    def initCells(self):
        
        if use_cuda:
            return Variable(torch.zeros(1, 1, hidden_size).cuda(gpu_no))
        else:
            return Variable(torch.zeros(1, 1, hidden_size))

In [77]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda(gpu_no)
        else:
            return result

In [83]:
def convert_target_to_tree(target_root,max_depth):
    
    #import pdb; pdb.set_trace()
    targetNodeQ = Queue()
    targetNodeQ.put(target_root)
    
    nodeQ = Queue()
    rootNode = Tree(0)
    nodeQ.put(rootNode)
     
    while(not targetNodeQ.empty()):
        target_node = targetNodeQ.get()
        node = nodeQ.get()
        
        if node.level == max_depth:
            break
        
        if(target_node.key in target_op):
            target_node.key = target_op[target_node.key]
        node.label = int(target_node.key)
        
        #Working with the assumption that tree has both left and right children. Because of the dataset
        if target_node.leftChild != None:
            
            if node.level+1 < max_depth:
                
                leftChild = Tree(node.level+1)
                node.leftChild = leftChild
            
                rightChild = Tree(node.level+1)
                node.rightChild = rightChild

                node.children.append(leftChild)
                node.children.append(rightChild)


                targetNodeQ.put(target_node.leftChild)
                targetNodeQ.put(target_node.rightChild)

                nodeQ.put(node.leftChild)
                nodeQ.put(node.rightChild)
       
    return rootNode   

In [92]:
def test(input_variable, target_variable, encoder, decoder, criterion):
    encoder_hidden = encoder.initCells()
    encoder_c = encoder.initCells()

    decoded_words = []
    
    input_length = len(input_variable)
    target_length = len(target_variable)

    loss = 0
    
    if use_cuda:
        encoder_hidden = encoder_hidden.cuda()

    for ei in range(input_length):
        encoder_output, encoder_hidden,encoder_c = encoder(input_variable[ei], encoder_hidden,encoder_c)

    decoder_hidden = encoder_hidden
    

    for di in range(200):
        decoder_output, decoder_hidden = decoder(
            target_variable[di], decoder_hidden)
        
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if int(ni) == 116:
            break
            
        if int(ni) in target_operations:
            ni = target_operations[int(ni)]
            
        if di != 0:
            decoded_words.append(str(ni))
            
        if di+1 < len(target_variable):
            loss += criterion(decoder_output, target_variable[di])    
        else:
            eos = Variable(torch.LongTensor([116]))
            loss += criterion(decoder_output, eos)

    return loss.data[0] / target_length,decoded_words

In [15]:
def testIters(encoder, decoder, n_iters):
    
    start = time.time()
    ted = 0  # Reset every print_every
    losses = 0
    criterion = nn.NLLLoss()
    

    for iter in range(1, n_iters + 1):
        test_pair = test_data[iter - 1]
        input_variable = Variable(torch.LongTensor(test_pair[0]).view(-1, 1))
        target_variable = Variable(torch.LongTensor(test_pair[1]).view(-1, 1))
        
        if use_cuda:
            input_variable = input_variable.cuda(gpu_no)
            target_variable = target_variable.cuda(gpu_no)
        
        loss,decoded_words = test(input_variable, target_variable, encoder, decoder, criterion)
        decoded_string = ' '.join(decoded_words)
        print(decoded_string)
        c = TreeConstructor()
        generated_tree = c.buildTree(decoded_string)
        generated = convert_target_to_tree(generated_tree,max_depth)
        target_tree = test_pair[2]
        t = convert_target_to_tree(target_tree,max_depth)
        ted_dist = simple_distance((generated), (t), Tree.get_children, Tree.get_label)
        ted += ted_dist
        print("Ted dist in ",iter, "is ",ted_dist)
        losses += loss
        
            
    print("Avergae tree edit distance: ",ted/1000)

In [16]:
hidden_size = 256
embedding_size = 256
gpu_no = 1
# 0-99,+,-,*,/,(,)
encoder_vocab_size = 113
decoder_vocab_size = 117
use_cuda = False

max_depth = 5

target_operations = {113:'(',114: ')', 109:'+', 110: '-', 111: '*', 112: '/'}
target_op = {'+':'109', '-':'110','*':'111','/':'112'}

encoder1 = EncoderLSTM(encoder_vocab_size, hidden_size,embedding_size)
decoder1 = DecoderRNN(hidden_size, decoder_vocab_size)

encoder1.load_state_dict(torch.load('./pickles/encoder_seq2seq.pth'))
decoder1.load_state_dict(torch.load('./pickles/decoder_seq2seq.pth'))

encoder1 = encoder1.eval()
decoder1 = decoder1.eval()

if use_cuda:
    encoder1 = encoder1.cuda(gpu_no)
    decoder1 = decoder1.cuda(gpu_no)
   

testIters(encoder1,decoder1, 1000)

( ( 18 - 45 ) * ( ( 75 * 89 ) + 88 ) )
Ted dist in  1 is  0.0
( ( 89 * ( 26 - ( 97 / ( 95 * 101 ) ) ) ) / ( 98 * ( 99 * ( 100 / 6 ) ) ) )
Ted dist in  2 is  0.0
( 51 + ( 95 * ( 64 + 96 ) ) )
Ted dist in  3 is  0.0
( ( ( 83 * 102 ) / 101 ) / ( ( ( 44 / 100 ) / ( 73 / ( 26 / 99 ) ) ) - 0 ) )
Ted dist in  4 is  0.0
( ( ( 100 * ( 101 * 33 ) ) / ( ( 63 - 99 ) + ( 21 - 98 ) ) ) * ( ( ( 97 - 23 ) * 16 ) + ( ( 95 * ( 81 * 96 ) ) + 94 ) ) )
Ted dist in  5 is  0.0
( ( ( 96 + ( ( 17 / 98 ) / 97 ) ) * ( ( 95 / 89 ) + 94 ) ) / ( ( 88 * ( 93 / 48 ) ) * ( 91 / ( 92 / 2 ) ) ) )
Ted dist in  6 is  0.0
( 45 - 44 )
Ted dist in  7 is  0.0
( 77 + 76 )
Ted dist in  8 is  0.0
( ( ( 59 * 100 ) + ( ( 84 - ( 34 + 99 ) ) + 29 ) ) - ( ( 97 * ( 98 * 23 ) ) / ( 96 - 25 ) ) )
Ted dist in  9 is  0.0
( ( 27 - ( 43 * 77 ) ) - 75 )
Ted dist in  10 is  0.0
( ( 91 + ( 6 - 92 ) ) / 90 )
Ted dist in  11 is  0.0
( ( ( 102 / ( 37 * ( 90 / 103 ) ) ) + ( 70 + ( 98 + 101 ) ) ) + ( ( ( 24 + 48 ) * 16 ) - ( 100 * 44 ) ) )
Ted dist

Ted dist in  90 is  0.0
( ( ( ( 99 * 40 ) * ( 30 + 5 ) ) / ( ( 97 - ( 98 / 88 ) ) * 96 ) ) / ( ( ( 38 - 95 ) * 94 ) + 6 ) )
Ted dist in  91 is  0.0
( ( ( ( 101 - 46 ) - ( ( 41 / 100 ) + 99 ) ) * 44 ) + ( ( 98 + 93 ) * 52 ) )
Ted dist in  92 is  0.0
( ( ( ( 70 - 104 ) - 103 ) / ( 62 - ( 66 / 102 ) ) ) * ( ( 44 / ( 79 - 101 ) ) * ( ( ( 100 / 42 ) - 99 ) * 92 ) ) )
Ted dist in  93 is  0.0
( 66 / 68 )
Ted dist in  94 is  0.0
( ( 19 / 74 ) + 73 )
Ted dist in  95 is  0.0
( ( 93 * 47 ) * ( ( 71 / ( 26 + 92 ) ) - ( 20 / 91 ) ) )
Ted dist in  96 is  0.0
( ( ( 101 * 37 ) - ( ( 92 + 100 ) + ( 78 * ( 91 - 99 ) ) ) ) * ( 96 * ( ( 98 * 1 ) + 97 ) ) )
Ted dist in  97 is  0.0
( ( 94 / 47 ) / 93 )
Ted dist in  98 is  0.0
( ( ( 103 / ( 104 / 91 ) ) / ( ( 32 - 102 ) / 101 ) ) - ( ( 70 - ( ( 3 + 100 ) + 59 ) ) * ( ( 57 * 99 ) - 7 ) ) )
Ted dist in  99 is  0.0
( ( ( ( 13 + 5 ) / 103 ) - ( ( 81 * 102 ) - 8 ) ) + ( ( ( 101 + 82 ) / ( ( 100 + 14 ) - 41 ) ) * ( 65 * ( 56 / 99 ) ) ) )
Ted dist in  100 is  0.0
(

( ( ( 10 / 99 ) / ( 96 - ( 97 + ( 89 - 98 ) ) ) ) - ( 39 / ( 95 * 72 ) ) )
Ted dist in  179 is  0.0
( ( 75 + ( ( 78 * 102 ) / ( 101 + 46 ) ) ) + ( ( 99 - ( 59 / 100 ) ) + ( ( 12 * ( 13 / 98 ) ) - 17 ) ) )
Ted dist in  180 is  0.0
( 55 * 99 )
Ted dist in  181 is  0.0
( ( ( 10 + 69 ) + ( 3 * ( 63 / 51 ) ) ) + ( ( 99 * 95 ) * ( ( ( 17 + 98 ) - 97 ) - 96 ) ) )
Ted dist in  182 is  0.0
( ( ( 95 - ( 98 - 39 ) ) + ( 61 + 97 ) ) / ( ( 88 / 96 ) * 21 ) )
Ted dist in  183 is  0.0
( ( ( ( 61 * 101 ) - ( 100 + 41 ) ) + ( ( 27 / 92 ) - ( 98 - ( 79 + 99 ) ) ) ) / ( 23 * ( 66 * 91 ) ) )
Ted dist in  184 is  0.0
( 27 * 93 )
Ted dist in  185 is  0.0
( 17 + 18 )
Ted dist in  186 is  0.0
( ( ( ( 98 / ( 52 * 99 ) ) * ( 21 - 72 ) ) - ( 73 / 17 ) ) - ( ( ( 44 * 97 ) / ( 43 * 96 ) ) / ( ( 81 / 95 ) / 23 ) ) )
Ted dist in  187 is  0.0
( ( ( 97 + ( 98 * ( 51 - 99 ) ) ) * ( ( 8 - 65 ) * ( 42 - 94 ) ) ) - ( ( 24 * 96 ) * ( ( 76 / 95 ) * 11 ) ) )
Ted dist in  188 is  0.0
( 81 + 80 )
Ted dist in  189 is  0.0
( ( (

( ( ( ( 55 * 95 ) + ( 94 - 83 ) ) - ( 33 * 18 ) ) - ( ( ( 93 + 57 ) / 13 ) / ( 19 + ( 91 + ( 92 * 68 ) ) ) ) )
Ted dist in  266 is  0.0
( ( 77 / 10 ) - ( ( ( 76 * 41 ) + 15 ) + 2 ) )
Ted dist in  267 is  0.0
( 88 * 30 )
Ted dist in  268 is  0.0
( 37 - 36 )
Ted dist in  269 is  0.0
( ( 68 - ( ( 104 * 73 ) / ( ( 103 / 99 ) * 84 ) ) ) - ( ( ( 102 + 59 ) + 101 ) / ( 40 * 100 ) ) )
Ted dist in  270 is  0.0
( ( ( ( 47 * 13 ) + ( 7 / 83 ) ) + ( 40 / ( 94 + ( 3 * 95 ) ) ) ) / ( ( 36 - 33 ) / ( 93 * 72 ) ) )
Ted dist in  271 is  0.0
( ( ( 94 * 10 ) + ( 93 / 14 ) ) / ( ( 79 - ( 92 + 51 ) ) / ( 62 * 76 ) ) )
Ted dist in  272 is  0.0
( ( 6 / ( 60 * ( 72 - ( 103 / 35 ) ) ) ) + ( ( ( 83 * 102 ) * 101 ) * ( ( 100 * 30 ) * ( 59 - 99 ) ) ) )
Ted dist in  273 is  0.0
( ( ( 95 / 92 ) * 94 ) / 93 )
Ted dist in  274 is  0.0
( ( 91 * ( ( 92 + 1 ) / 59 ) ) - ( 11 / ( ( ( 90 * 88 ) - 89 ) / 9 ) ) )
Ted dist in  275 is  0.0
( ( ( ( 97 - 102 ) - ( 51 - ( 101 - 2 ) ) ) + ( ( 41 * 100 ) - 13 ) ) + ( ( 92 + 99 ) /

( ( ( 103 - ( 86 + 104 ) ) * ( ( 102 * 76 ) / ( 100 - ( 101 + 27 ) ) ) ) * ( ( 99 / 50 ) * ( 91 * ( 68 + 98 ) ) ) )
Ted dist in  356 is  0.0
( ( 84 - ( ( ( 69 * 94 ) - 93 ) - 11 ) ) * ( ( ( 92 + 7 ) * 91 ) - ( ( 63 + 80 ) - 49 ) ) )
Ted dist in  357 is  0.0
( ( ( ( 84 * 105 ) - 104 ) / ( ( 102 * ( 71 - 103 ) ) + 101 ) ) * ( ( 45 + ( 4 * 100 ) ) / ( 25 / 99 ) ) )
Ted dist in  358 is  0.0
( ( ( 91 - ( 61 + 93 ) ) * ( 5 * ( 102 * 59 ) ) ) / ( ( ( ( 79 * 101 ) + 100 ) + ( 71 / 52 ) ) * ( ( 99 * 3 ) + ( 47 / 28 ) ) ) )
Ted dist in  359 is  0.0
( ( ( ( 101 - 34 ) / 4 ) / 56 ) + ( 65 - ( 100 * 83 ) ) )
Ted dist in  360 is  0.0
( ( 73 / 21 ) * ( ( 98 / ( 99 * ( 100 + 39 ) ) ) - 17 ) )
Ted dist in  361 is  0.0
( ( ( ( 34 * 103 ) + 81 ) - 102 ) - ( ( ( 73 / 101 ) - 15 ) - ( ( 99 + ( 49 * 100 ) ) - ( 98 * 72 ) ) ) )
Ted dist in  362 is  0.0
( ( ( 62 / 46 ) + 61 ) + 60 )
Ted dist in  363 is  0.0
( ( 60 - 93 ) * ( ( ( 97 / 49 ) * 40 ) / 96 ) )
Ted dist in  364 is  0.0
( ( ( 103 + ( 21 * 104 ) ) + (

...

( ( ( 39 / 80 ) * ( ( ( 48 - 99 ) * 98 ) + ( 97 * 76 ) ) ) * ( ( ( 96 + 36 ) * 24 ) - ( 95 - 84 ) ) )
Ted dist in  615 is  0.0
( ( ( 97 * ( 98 * 1 ) ) - 96 ) * ( 95 * 78 ) )
Ted dist in  616 is  0.0
( ( ( ( 99 - 58 ) * ( 98 - 1 ) ) * ( 96 - ( 50 + 97 ) ) ) + ( ( 26 * ( 94 / ( 95 * 42 ) ) ) / 13 ) )
Ted dist in  617 is  0.0
( ( ( 92 * ( 20 - 78 ) ) + ( 102 + 33 ) ) / ( ( 55 / ( 14 + ( 86 + 101 ) ) ) * ( ( 95 / 100 ) - 99 ) ) )
Ted dist in  618 is  0.0
( ( ( 14 - 101 ) / ( ( 99 / ( 94 + 100 ) ) / ( 19 + 98 ) ) ) - ( ( ( 97 - 31 ) / 47 ) + ( ( 56 - 96 ) / ( 61 / 95 ) ) ) )
Ted dist in  619 is  0.0
( ( ( 99 / 81 ) - ( 8 / ( 98 * 95 ) ) ) / ( 57 / ( ( ( 97 + 92 ) + 96 ) * 51 ) ) )
Ted dist in  620 is  0.0
( ( ( ( ( 15 - 94 ) - 93 ) - ( 3 * 28 ) ) / ( ( 33 / 92 ) - 70 ) ) * ( ( ( 55 - 89 ) + ( 91 - 52 ) ) * 85 ) )
Ted dist in  621 is  0.0
( ( 21 - ( ( 11 * 5 ) - 57 ) ) * ( ( ( 4 * 99 ) * 22 ) * ( 97 + ( 60 * ( 98 * 77 ) ) ) ) )
Ted dist in  622 is  0.0
( ( 89 * ( ( 94 * ( 95 - 1 ) ) / 27 ) )

( ( ( 15 / 76 ) / ( ( 99 * 47 ) + 98 ) ) / ( ( 97 * 88 ) * 96 ) )
Ted dist in  701 is  0.0
( ( ( ( 44 * 106 ) + 105 ) - ( 104 + 42 ) ) / ( ( ( 103 + 11 ) * 102 ) - ( ( ( 101 * 86 ) - 37 ) * 100 ) ) )
Ted dist in  702 is  0.0
( ( ( 91 * ( 22 + 92 ) ) * 66 ) * ( ( ( ( 53 + 90 ) / 89 ) - 31 ) / ( 88 + 67 ) ) )
Ted dist in  703 is  0.0
( ( ( 50 + 57 ) * ( ( 31 * 102 ) - 59 ) ) * ( ( ( ( 101 / 24 ) - 100 ) * 17 ) + ( ( 47 + 99 ) * 98 ) ) )
Ted dist in  704 is  0.0
( ( ( ( 107 - 26 ) + 106 ) * ( 104 + ( 44 * 105 ) ) ) + ( 100 / ( ( 102 * ( 90 - 103 ) ) / 101 ) ) )
Ted dist in  705 is  0.0
( ( ( ( 103 * 14 ) + ( 102 / 50 ) ) - 84 ) - ( ( ( 100 + ( 12 * 101 ) ) + 99 ) / ( 98 / 10 ) ) )
Ted dist in  706 is  0.0
( ( ( 38 / 94 ) / ( 21 * ( 78 / 93 ) ) ) - ( ( 25 * ( ( 92 / 69 ) - 91 ) ) * ( 52 / ( 86 + 5 ) ) ) )
Ted dist in  707 is  0.0
( ( ( 103 * ( 104 * 19 ) ) + ( ( ( 102 - 14 ) + 101 ) - ( 28 - 100 ) ) ) / ( 98 * ( 99 - 42 ) ) )
Ted dist in  708 is  0.0
( ( 34 - ( 36 - 87 ) ) / ( ( ( ( 103 - 

Ted dist in  787 is  0.0
( ( ( 25 - 94 ) * 90 ) + ( 60 - 93 ) )
Ted dist in  788 is  0.0
( 68 / ( ( 103 + 93 ) * ( ( ( 102 / 99 ) - 101 ) / 100 ) ) )
Ted dist in  789 is  0.0
( ( ( ( 91 / ( 92 * 71 ) ) - 39 ) / ( ( 90 + 2 ) + ( 89 * 67 ) ) ) + ( ( 88 + 41 ) / ( ( 87 + 21 ) * 84 ) ) )
Ted dist in  790 is  0.0
( ( 100 * 6 ) - ( ( ( 33 - 0 ) + ( ( 96 * 99 ) - 51 ) ) / ( ( 98 - 90 ) * ( 97 / 69 ) ) ) )
Ted dist in  791 is  0.0
( ( ( ( 64 * ( 95 - 50 ) ) / ( 94 - 17 ) ) / ( 7 / ( 93 - 63 ) ) ) + ( ( ( 4 * 92 ) / 91 ) + 90 ) )
Ted dist in  792 is  0.0
( ( ( 101 + 33 ) + 100 ) / ( ( ( 50 / ( 99 - 72 ) ) - 55 ) + ( 97 / 98 ) ) )
Ted dist in  793 is  0.0
( ( ( 70 + 61 ) / ( 94 * ( ( 96 / 87 ) + 95 ) ) ) + ( 92 + ( 93 / 19 ) ) )
Ted dist in  794 is  0.0
( 25 + 26 )
Ted dist in  795 is  0.0
( ( ( 8 / 61 ) + ( 100 * ( 101 + ( 102 - 28 ) ) ) ) + ( 98 / ( 99 - 97 ) ) )
Ted dist in  796 is  0.0
( ( ( ( 71 * 8 ) + 103 ) * ( ( 101 * ( 102 + 11 ) ) + ( 24 + 100 ) ) ) / ( 72 + 69 ) )
Ted dist in  797 is 

( ( 87 + 14 ) + ( ( 86 / 85 ) + 0 ) )
Ted dist in  882 is  0.0
( 58 / ( 69 / 19 ) )
Ted dist in  883 is  0.0
( ( ( 86 / 91 ) / ( ( ( 90 - 5 ) - 89 ) - 88 ) ) - ( 2 + ( 44 * 87 ) ) )
Ted dist in  884 is  0.0
( ( 90 / ( 91 + 42 ) ) * 3 )
Ted dist in  885 is  0.0
( ( ( ( 21 * ( 48 / 96 ) ) / 78 ) * ( 94 / ( 95 * 16 ) ) ) * ( 92 / ( 88 / 40 ) ) )
Ted dist in  886 is  0.0
( ( ( ( 96 * 101 ) + ( 99 + ( 100 * 82 ) ) ) - 66 ) + ( 9 + 20 ) )
Ted dist in  887 is  0.0
( ( 94 * ( 21 - 95 ) ) * 93 )
Ted dist in  888 is  0.0
( ( ( ( ( 101 + 87 ) - 100 ) - ( 50 - 99 ) ) - ( 95 + 98 ) ) + ( ( 97 + 59 ) * ( 96 + 89 ) ) )
Ted dist in  889 is  0.0
( ( ( 36 - ( 75 / 99 ) ) * ( 5 * 98 ) ) / ( ( 93 + 90 ) * ( ( 71 / 97 ) / ( 38 - ( 40 / 96 ) ) ) ) )
Ted dist in  890 is  0.0
( ( ( ( 91 - 105 ) * ( ( 104 * 96 ) + 103 ) ) - ( 40 + 102 ) ) * ( ( 100 * ( 101 + 73 ) ) / ( 52 / 36 ) ) )
Ted dist in  891 is  0.0
( 35 + 36 )
Ted dist in  892 is  0.0
( 22 + 10 )
Ted dist in  893 is  0.0
( ( ( ( 102 - ( 103 / 82 ) ) *

( ( ( ( ( 57 + 102 ) * 101 ) + 41 ) - ( 69 - ( 100 * 26 ) ) ) + ( ( 15 + 99 ) / ( 46 / 52 ) ) )
Ted dist in  973 is  0.0
( 96 - 95 )
Ted dist in  974 is  0.0
( ( ( 28 / 83 ) + ( 49 - ( ( 82 / 81 ) / 3 ) ) ) - ( 61 * 74 ) )
Ted dist in  975 is  0.0
( ( 6 * ( ( 36 + 13 ) + 92 ) ) - ( ( 68 * ( ( 80 - 91 ) - 90 ) ) - ( ( 63 - 89 ) - 11 ) ) )
Ted dist in  976 is  0.0
( 35 + ( 36 + ( 37 + 34 ) ) )
Ted dist in  977 is  0.0
( ( ( 82 - 96 ) - ( ( 70 - 42 ) - 67 ) ) + ( ( ( 95 + 41 ) + ( ( 43 * 94 ) * 87 ) ) * ( 21 + 51 ) ) )
Ted dist in  978 is  0.0
( ( ( 103 + 73 ) + ( ( 80 + 24 ) / 102 ) ) / ( ( ( ( 87 * 101 ) - 100 ) - 99 ) * 98 ) )
Ted dist in  979 is  0.0
( 56 + ( 55 - 57 ) )
Ted dist in  980 is  0.0
( ( ( 93 * ( 12 - 94 ) ) - ( ( 22 * 64 ) + ( 91 + ( 32 - 92 ) ) ) ) / ( ( 90 - 27 ) * ( 36 - 89 ) ) )
Ted dist in  981 is  0.0
( ( ( 69 - 70 ) - 65 ) + 61 )
Ted dist in  982 is  0.0
( 97 - ( 98 * 17 ) )
Ted dist in  983 is  0.0
( ( 66 - 86 ) * ( ( 60 / 85 ) - 65 ) )
Ted dist in  984 is  0.0
( 